# Experiment SetUp - 50 Trials

Parameters:
- l1
- l1 vs. batch_size
- epochs vs. batch_size
- epochs vs. lr
- lr vs. batch_size

### Imports

In [11]:
from functools import partial
import os
import random 

import matplotlib.pyplot as plt
import numpy as np

import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import random_split

from ray import tune
from ray.air import Checkpoint, session
from ray.tune.schedulers import ASHAScheduler
from ray.tune.search.basic_variant import BasicVariantGenerator
from ray.tune import Callback

In [12]:
torch.manual_seed(40)
random.seed(40)
np.random.seed(40)

#### Class to print chosen l1 param

In [13]:
class PrintChosenL1Callback(Callback):
    def on_trial_complete(self, iteration, trials, trial, **info):
        chosen_l1 = trial.config["l1"]
        print(f"Trial {trial.trial_id}: Chosen l1 = {chosen_l1}")

In [14]:
print_l1_callback = PrintChosenL1Callback()

### CNN Model SetUp

In [15]:
class FashionCNN(nn.Module):
    def __init__(self, l1=64):
        super(FashionCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 7 * 7, l1)
        self.fc2 = nn.Linear(l1, 10)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

### Training

#### Training function

In [16]:
def train_fashion_mnist(config):
    net = FashionCNN(config["l1"]) 

    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.to(device)

    # loading data
    data_dir = "./data"
    transform = transforms.Compose(
        [transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))])

    trainset = torchvision.datasets.FashionMNIST(data_dir,
        download=True,
        train=True,
        transform=transform)
    testset = torchvision.datasets.FashionMNIST(data_dir,
        download=True,
        train=False,
        transform=transform)
    
    # Split the training set into subsets
    test_abs = int(len(trainset) * 0.8)
    train_subset, val_subset = random_split(trainset, [test_abs, len(trainset) - test_abs])

    # Create data loaders for subsets
    trainloader = torch.utils.data.DataLoader(
        train_subset, batch_size=config["batch_size"], shuffle=True, num_workers=2
    )
    valloader = torch.utils.data.DataLoader(
        val_subset, batch_size=config["batch_size"], shuffle=True, num_workers=2
    )

    # defining the loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=config["learning_rate"])

    for epoch in range(config["epochs"]):
        running_loss = 0.0
        epoch_steps = 0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            epoch_steps += 1

            if i % 2000 == 1999:
                print("[%d, %5d] loss: %.3f" % (epoch + 1, i + 1, running_loss / epoch_steps))
                running_loss = 0.0

        val_loss = 0.0
        val_steps = 0
        total = 0
        correct = 0
        with torch.no_grad():
            for data in valloader:
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                loss = criterion(outputs, labels)
                val_loss += loss.item()
                val_steps += 1

        tune.report(
            mean_accuracy=correct / total,
            mean_val_loss=val_loss / val_steps
        )

#### Experiment Parameters

In [17]:
max_num_epochs = 15
num_samples = 50

#### AHSA scheduler

In [18]:
scheduler = ASHAScheduler(
    metric="mean_val_loss",
    mode="min",
    max_t=max_num_epochs,
    grace_period=1,
    reduction_factor=2,
)

### 1st Experiment - l1

In [19]:
config = {
    "epochs": tune.choice([10]),
    "learning_rate": 1e-3, # 0.001
    "batch_size": tune.choice([64]),
    "l1": tune.sample_from(lambda _: 2**np.random.randint(5, 7)) # 32, 64, 128
}

In [20]:
result_l1 = tune.run(
    partial(train_fashion_mnist),
    resources_per_trial={"cpu": 8, "gpu": 0},
    config=config,
    num_samples=num_samples,
    storage_path='./tune_runs/',
    search_alg=BasicVariantGenerator(random_state=40),
    scheduler=scheduler,
    callbacks=[print_l1_callback]
)

best_trial = result_l1.get_best_trial("mean_val_loss", mode="min")
best_config = best_trial.config
chosen_l1 = best_config["l1"]

print("Chosen l1:", chosen_l1)

best_metrics = best_trial.metric_analysis

print("Best trial config:", best_config)
print("Best trial metrics:", best_metrics)

2023-08-21 16:46:56,520	INFO tune.py:657 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


(func pid=42544) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz
(func pid=42544) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz to ./data\FashionMNIST\raw\train-images-idx3-ubyte.gz


0.5%(func pid=42544) 
1.2%(func pid=42544) 
1.4%(func pid=42544) 
2.4%(func pid=42544) 
2.5%(func pid=42544) 
3.3%(func pid=42544) 
4.3%(func pid=42544) 
5.5%(func pid=42544) 
6.4%(func pid=42544) 
6.6%(func pid=42544) 
7.6%(func pid=42544) 
7.7%(func pid=42544) 
8.6%(func pid=42544) 
8.7%(func pid=42544) 
8.8%(func pid=42544) 
9.8%(func pid=42544) 
10.7%(func pid=42544) 
11.7%(func pid=42544) 
12.5%(func pid=42544) 
13.3%(func pid=42544) 
13.4%(func pid=42544) 
14.3%(func pid=42544) 
14.4%(func pid=42544) 
15.1%(func pid=42544) 
15.9%(func pid=42544) 
16.9%(func pid=42544) 
17.6%(func pid=42544) 
18.6%(func pid=42544) 
19.5%(func pid=42544) 
20.2%(func pid=42544) 
20.3%(func pid=42544) 
21.0%(func pid=42544) 
21.8%(func pid=42544) 
22.6%(func pid=42544) 
22.7%(func pid=42544) 
23.7%(func pid=42544) 
24.6%(func pid=42544) 
25.2%(func pid=42544) 
25.3%(func pid=42544) 
26.0%(func pid=42544) 
26.9%(func pid=42544) 
27.7%(func pid=42544) 
28.5%(func pid=42544) 
29.5%(func pid=42544) 
30.3

(func pid=42544) Extracting ./data\FashionMNIST\raw\train-images-idx3-ubyte.gz to ./data\FashionMNIST\raw
(func pid=42544) 
(func pid=42544) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz
(func pid=42544) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz to ./data\FashionMNIST\raw\train-labels-idx1-ubyte.gz
(func pid=42544) Extracting ./data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to ./data\FashionMNIST\raw
(func pid=42544) 
(func pid=42544) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-images-idx3-ubyte.gz


100.0%36m(func pid=42544) 


...

(func pid=42544) Extracting ./data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to ./data\FashionMNIST\raw
(func pid=42544) 
(func pid=42544) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-labels-idx1-ubyte.gz
(func pid=42544) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-labels-idx1-ubyte.gz to ./data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz


100.0%36m(func pid=42544) 


(func pid=42544) Extracting ./data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\FashionMNIST\raw
(func pid=42544) 


2023-08-21 16:58:15,244	WARNING tune.py:192 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2023-08-21 16:58:25,564	INFO tune.py:1148 -- Total run time: 689.04 seconds (678.70 seconds for the tuning loop).
2023-08-21 16:58:25,565	WARNING tune.py:1163 -- Experiment has been interrupted, but the most recent state was saved.
Resume experiment with: tune.run(..., resume=True)
2023-08-21 16:58:25,595	WARNING experiment_analysis.py:916 -- Failed to read the results for 17 trials:
- C:\Users\ibele\ray_results\train_fashion_mnist_2023-08-21_16-46-56\train_fashion_mnist_93602_00001_1_batch_size=64,epochs=10_2023-08-21_16-46-56
- C:\Users\ibele\ray_results\train_fashion_mnist_2023-08-21_16-46-56\train_fashion_mnist_93602_00002_2_batch_size=64,epochs=10_2023-08-21_16-46-56
- C:\Users\ibele\ray_results\train_fashion_mnist_2023-08-21_16-46-56\train_fashion_m

Chosen l1: 32
Best trial config: {'epochs': 10, 'learning_rate': 0.001, 'batch_size': 64, 'l1': 32}
Best trial metrics: {'mean_accuracy': {'max': 0.7890833333333334, 'min': 0.41508333333333336, 'avg': 0.7185416666666669, 'last': 0.7855, 'last-5-avg': 0.7798499999999999, 'last-10-avg': 0.7185416666666666}, 'mean_val_loss': {'max': 2.1220153164356312, 'min': 0.567176804897633, 'avg': 0.8612084210077496, 'last': 0.567176804897633, 'last-5-avg': 0.5957349429422236, 'last-10-avg': 0.8612084210077497}, 'time_this_iter_s': {'max': 71.91051888465881, 'min': 53.20462203025818, 'avg': 62.499757170677185, 'last': 67.07591915130615, 'last-5-avg': 67.05286979675293, 'last-10-avg': 62.499757170677185}, 'done': {'max': False, 'min': False, 'avg': 0.0, 'last': False, 'last-5-avg': 0.0, 'last-10-avg': 0.0}, 'training_iteration': {'max': 10, 'min': 1, 'avg': 5.5, 'last': 10, 'last-5-avg': 8.0, 'last-10-avg': 5.5}, 'time_total_s': {'max': 624.9975717067719, 'min': 70.44486260414124, 'avg': 335.6692216873

### 2nd Experiment - l1 vs. batch_size

In [ ]:
config = {
    "epochs": tune.choice([10]),
    "learning_rate": tune.loguniform(1e-3), 
    "batch_size": tune.choice([16, 32, 64, 128]),
    "l1": tune.sample_from(lambda _: 2**np.random.randint(5, 7))
}

In [ ]:
result_l1_bs = tune.run(
    partial(train_fashion_mnist),
    resources_per_trial={"cpu": 8, "gpu": 0},
    config=config,
    num_samples=num_samples,
    storage_path='./tune_runs/',
    search_alg=BasicVariantGenerator(random_state=40),
    scheduler=scheduler,
    callbacks=[print_l1_callback]
)

best_trial = result_l1_bs.get_best_trial("mean_val_loss", mode="min")
best_config = best_trial.config
chosen_l1 = best_config["l1"]

print("Chosen l1:", chosen_l1)

best_metrics = best_trial.metric_analysis

print("Best trial config:", best_config)
print("Best trial metrics:", best_metrics)

### 3rd Experiment - epochs vs. batch_size

In [ ]:
config = {
    "epochs": tune.choice([5, 10, 15]),
    "learning_rate": tune.loguniform(1e-3), 
    "batch_size": tune.choice([16, 32, 64, 128])
}

In [ ]:
result_ep_bs = tune.run(
    partial(train_fashion_mnist),
    resources_per_trial={"cpu": 8, "gpu": 0},
    config=config,
    num_samples=num_samples,
    storage_path='./tune_runs/',
    search_alg=BasicVariantGenerator(random_state=40),
    scheduler=scheduler
)

best_trial = result_ep_bs.get_best_trial("mean_val_loss", mode="min")
best_config = best_trial.config
chosen_l1 = best_config["l1"]

print("Chosen l1:", chosen_l1)

best_metrics = best_trial.metric_analysis

print("Best trial config:", best_config)
print("Best trial metrics:", best_metrics)

### 4th Experiment - epochs vs. lr

In [ ]:
config = {
    "epochs": tune.choice([5, 10, 15]),
    "learning_rate": tune.loguniform(1e-3, 1e-1), 
    "batch_size": tune.choice([64])
}

In [ ]:
result_ep_lr = tune.run(
    partial(train_fashion_mnist),
    resources_per_trial={"cpu": 8, "gpu": 0},
    config=config,
    num_samples=num_samples,
    storage_path='./tune_runs/',
    search_alg=BasicVariantGenerator(random_state=40),
    scheduler=scheduler
)

best_trial = result_ep_lr.get_best_trial("mean_val_loss", mode="min")
best_config = best_trial.config
chosen_l1 = best_config["l1"]

print("Chosen l1:", chosen_l1)

best_metrics = best_trial.metric_analysis

print("Best trial config:", best_config)
print("Best trial metrics:", best_metrics)

### 5th Experiment - batch_size vs. lr

In [ ]:
config = {
    "epochs": tune.choice([10]),
    "learning_rate": tune.loguniform(1e-3, 1e-1), 
    "batch_size": tune.choice([16, 32, 64, 128])
}

In [ ]:
result_bs_lr = tune.run(
    partial(train_fashion_mnist),
    resources_per_trial={"cpu": 8, "gpu": 0},
    config=config,
    num_samples=num_samples,
    storage_path='./tune_runs/',
    search_alg=BasicVariantGenerator(random_state=40),
    scheduler=scheduler
)

best_trial = result_bs_lr.get_best_trial("mean_val_loss", mode="min")
best_config = best_trial.config
chosen_l1 = best_config["l1"]

print("Chosen l1:", chosen_l1)

best_metrics = best_trial.metric_analysis

print("Best trial config:", best_config)
print("Best trial metrics:", best_metrics)